In [1]:
import sys, os
import torch
#import librosa
import numpy as np
import pandas as pd
from torch import Tensor
from scipy.io import wavfile
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchaudio
import numpy as np
import scipy.io.wavfile
import torch.nn.functional as F



In [2]:
class RawNetDataset(Dataset):
    def __init__(self, transform=None, mode="train",files_dir=None, base_dir="",csv_file_dir="",nb_time=59049):
        self.base_dir = base_dir
        self.mode = mode
        self.transform = transform
        self.csv_file_dir = csv_file_dir
        self.files_dir = files_dir
        csv_file = pd.read_csv(csv_file_dir,sep="\t")
        self.nb_time = nb_time
        #self.classes = {cls_name:i+1 for i, cls_name in enumerate(csv_file["VoxCeleb1 ID"].unique())}
        self.classes = {'id10009': 0,'id10016': 1,'id10017': 2,'id10019': 3}

    def __len__(self):
        
        return len(self.files_dir)
    
    def __getitem__(self, idx):
        filename = self.files_dir[idx]
        classe = filename.split('/')[0]
        X, sample_rate = torchaudio.load(self.base_dir + filename)
        #print(" shape(X): ",type(X))
        label = self.classes[classe]
        self._pre_emphasis(X)
        nb_time = X.shape[1]
        if nb_time > self.nb_time:
            start_idx = np.random.randint(low = 0,
                high = nb_time - self.nb_time)
            X = X[:, start_idx:start_idx+self.nb_time]
            #print("nb_time: ",nb_time )
            #print("self.nb_time: ",self.nb_time)
        elif nb_time < self.nb_time:
            nb_dup = int(self.nb_time / nb_time) + 1
            X = np.tile(X, (1, nb_dup))[:, :self.nb_time]
            #print("taille inférieure")
        else:
            X = X
            #print("taille égale")
        #print(" type(X): ",X.size())
        return X, label

    def _pre_emphasis(self, x):
        '''
        Pre-emphasis for single channel input
        '''
        return np.asarray(x[:,1:] - 0.97 * x[:, :-1], dtype=np.float32) 



In [3]:
base_dir = 'data/wav/'
csv_file_dir = 'data/target/vox1_meta.csv'
def get_utt_list(src_dir):
    l_utt = []
    for r, ds, fs in os.walk(src_dir):
        r = r.replace('\\', '/')   
        base = '/'.join(r.split('/')[-2:])+'/'
        for f in fs:
            l_utt.append(base+f[:-4]+'.wav')
    return l_utt
list_IDs = get_utt_list(base_dir)

In [4]:

input_size = 59049       # input size
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 4      # The number of output classes. In this case, from 0 to 9
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

In [5]:
#TRAIN DATA
train_dataset = RawNetDataset(files_dir=list_IDs,base_dir=base_dir,csv_file_dir=csv_file_dir)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [32]:
print(type(train_dataset.__getitem__(0)[0]))

<class 'torch.Tensor'>


In [6]:
#TEST DATA
test_dataset = RawNetDataset(files_dir=list_IDs,base_dir=base_dir,csv_file_dir=csv_file_dir)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        #print(out)
        out = F.softmax(out, dim=0)
        return out

In [9]:
net = Net(input_size, hidden_size, num_classes)

In [10]:
criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [24]:
for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(train_loader):   # Load a batch of audio with its (index, data, class)
        
        #print('data.shape',data.shape)
        #print('labels.size',labels.shape)
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(data)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs.squeeze(), labels.long())                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % num_epochs == 0:                              # Logging
            #print(loss.item())
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
Epoch [1/5], Step [5/5], Loss: 1.3773
data.shape torch.Size([42, 1, 59049])
labels.size torch.Size([42])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
Epoch [2/5], Step [5/5], Loss: 1.3749
data.shape torch.Size([42, 1, 59049])
labels.size torch.Size([42])
data.shape torch.Size([100, 1, 59049])
labels.size torch.Size([100])
data.shape torch.Size([100, 1, 

In [ ]:
print('ok2')

In [13]:
sequence_length   = 75
number_of_classes = 55
# creates random tensor of your output shape
output = torch.rand(sequence_length, 1, number_of_classes)
# creates tensor with random targets
target = torch.randint(55, (75,)).long()

# define loss function and calculate loss
criterion = nn.CrossEntropyLoss()

# apply squeeze() on output tensor to change shape form [75, 1, 55] to [75, 55]
loss = criterion(output.squeeze(), target)
print(output.size())

torch.Size([75, 1, 55])


In [14]:
# init CE Loss function
criterion = nn.CrossEntropyLoss()

# sequence of length 1
output = torch.rand(1, 5)
# in this case the 1th class is our target, index of 1th class is 0
target = torch.LongTensor([0])
loss = criterion(output, target)
print('Sequence of length 1:')
print('Output:', output, 'shape:', output.shape)
print('Target:', target, 'shape:', target.shape)
print('Loss:', loss)

Sequence of length 1:
Output: tensor([[0.1582, 0.5060, 0.3160, 0.7548, 0.4619]]) shape: torch.Size([1, 5])
Target: tensor([0]) shape: torch.Size([1])
Loss: tensor(1.9107)
